In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tik
from datetime import datetime, timezone, timedelta

from scipy.optimize import curve_fit
from scipy.stats import norm
import matplotlib.mlab as mlab

#from matplotlib import gridspec

# Input data, log and these forms

In [ ]:
#file path and name
filepath = './rawdata/'
filename = 'Jan19_absolute2.dat'

logpath = './log/'
logfilename = 'PMX_2021-01-19.dat'

In [ ]:
storepath = './plots/'
storename = 'Jan19_absolute2'

In [ ]:
#rawdata
time_category0, error0, direction0, timercount0, reference0 = np.loadtxt(filepath + filename, comments='#', unpack=True)

In [ ]:
#log
df0 = pd.read_csv(logpath + logfilename, delim_whitespace=True, comment='#')

In [ ]:
start_at = '2021-01-19 17:17:05-UTC'

In [ ]:
iteration = 640

list_amp = [3.0]
list_intv = ['0.401']

matrix_size = 1

pack_size = 1

In [ ]:
start_slice = np.where(start_at == df0.iloc[:,0].to_numpy()+' '+df0.iloc[:,1].to_numpy())[0][0]

In [ ]:
df = df0[start_slice:]
df

In [ ]:
position_log = (df.iloc[:,9][df.iloc[:,9] != "--------"])[(df.iloc[:,8][df.iloc[:,9] != "--------"]) == list_intv[0]].astype(float).to_numpy()
position_log

In [ ]:
min(error0)

# Set variables and checks

In [ ]:
tape_counts = 52000
PRU_clocks = 200e6

Deg = 360/tape_counts
invDeg = tape_counts/360

In [ ]:
time = timercount0/PRU_clocks

In [ ]:
count_MAX = 62000

## shape and data loss check

In [ ]:
#first scketch of the rawdata
fig01, axarr01 = plt.subplots(1,2,figsize=(14,7))
ax01 = axarr01.ravel()

ax01[0].plot(time[::100], reference0[::100])
ax01[0].set_xlabel('time[sec]')
ax01[0].set_ylabel('reference count')

ax01[1].scatter(np.diff(timercount0), np.diff(reference0))
ax01[1].set_xlabel('difference time of lines')
ax01[1].set_ylabel('difference counts of lines')

In [ ]:
error_UnixTime = np.array(time_category0[np.where((np.diff(timercount0) > 110000) | (np.diff(timercount0) < 90000))[0]], dtype=int)

error_Time = time[:-1][(np.diff(timercount0) > 110000) | (np.diff(timercount0) < 90000)]
error_TimeDelta = np.diff(time)[(np.diff(timercount0) > 110000) | (np.diff(timercount0) < 90000)]

In [ ]:
for i in range(len(error_Time)):
    print('{:8.2f} {:5.4f}'.format(round(error_Time[i],2), round(error_TimeDelta[i],5)))

In [ ]:
plt.scatter(error_Time, error_TimeDelta,s=10)

## set variables

In [ ]:
capture_slice = 1000
plot_slice = 30

pos_roll = 3

casp_threshold = -100

In [ ]:
#log slicing with JST
def packet_capture(dataframe, list_ampere, list_interval, UnixTime_Data):
    JST = timezone(timedelta(hours=+9), 'JST')
    initial_UnixTime = []
    switching_UnixTime = []
    start_UnixTime = []

    for i in range(iteration):
        start_UnixTime.append([])

    for i in list_ampere:
        for j in list_interval:
            list_datetime = ((dataframe[dataframe.iloc[:,5] == i])[(dataframe[dataframe.iloc[:,5] == i]).iloc[:,8] == j]).iloc[:,0:2].to_numpy()
            for k in range(iteration):
                if k%pack_size == 0:
                    start_UnixTime[int(k/pack_size)] = (((datetime.strptime(list_datetime[k][0]+' '+list_datetime[k][1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()
                    initial_UnixTime = np.append(initial_UnixTime, start_UnixTime[int(k/pack_size)])
                    pass
                pass
            switching_UnixTime.append(min(start_UnixTime))
            pass
        pass

    stop_UnixTime = dataframe[dataframe.iloc[:,2] == 'OFF'].iloc[-1,:]
    end_UnixTime = (((datetime.strptime(stop_UnixTime[0]+' '+stop_UnixTime[1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()

    captured_initials = []
    captured_switching = []
    
    for i in range(len(switching_UnixTime)):
        captured_switching.append(max(np.where(UnixTime_Data[::capture_slice] <= switching_UnixTime[i])[0]))
        pass
    
    for i in range(len(initial_UnixTime)):
        captured_initials.append(max(np.where(UnixTime_Data[::capture_slice] <= initial_UnixTime[i])[0]))
        pass
    
    captured_end = min(np.where(end_UnixTime + pos_roll <= UnixTime_Data[::capture_slice])[0])
    
    return captured_switching, captured_initials, captured_end

In [ ]:
swt, inits, end = packet_capture(df, list_amp, list_intv, time_category0)

In [ ]:
reference = reference0

for i in np.where(reference0 > 52000)[0]:
    reference[i] = reference0[i]-count_MAX

In [ ]:
#block
time_block = []
reference_block = []

for i in range(len(swt)-1):
    time_block.append(time[int(np.where(time == (time[::capture_slice])[swt[i]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[swt[i+1]-pos_roll])[0])])
    reference_block.append(reference[int(np.where(time == (time[::capture_slice])[swt[i]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[swt[i+1]-pos_roll])[0])])
    pass
time_block.append(time[int(np.where(time == (time[::capture_slice])[swt[-1]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[end+pos_roll])[0])])
reference_block.append(reference[int(np.where(time == (time[::capture_slice])[swt[-1]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[end+pos_roll])[0])])

lincount_block = []
block_initials = []
onecycle_time = []
proceeded_degrees = []

for i in range(matrix_size):
    lincount_block.append([])
    lincount_block[i] = np.array(np.zeros(len(reference_block[i])))
    
    count_offset = 0
    casp_flag = 0
    casp = np.where(np.diff(reference_block[i]) <= casp_threshold)[0].tolist()

    for j in range(len(reference_block[i])):
        lincount_block[i][j] = reference_block[i][j]+count_offset
        if casp:
            if j == casp[casp_flag]:
                count_offset += reference_block[i][casp[casp_flag]]
                if len(casp) > casp_flag+1:
                    casp_flag += 1
                    pass
                pass
            pass
        pass
    
    block_initials.append([])
    for j in range(len(inits)):
        block_initials[i].append(int(np.where(time_block[i] == (time[::capture_slice])[(inits[i*iteration:(i+1)*iteration])[j]-pos_roll])[0]))
        pass
    
    onecycle_time.append([])
    proceeded_degrees.append([])
    for j in range(len(inits) - 1):
        if j != len(inits) - 1:
            onecycle_time[i].append(time_block[i][block_initials[i][j+1]]-time_block[i][block_initials[i][j]])
            proceeded_degrees[i].append((lincount_block[i][block_initials[i][j+1]]-lincount_block[i][block_initials[i][j]])*Deg)
            pass
        else:
            onecycle_time[i].append(time_block[i][-1]-time_block[i][block_initials[i][j]])
            proceeded_degrees[i].append((lincount_block[i][-1]-lincount_block[i][block_initials[i][j]])*Deg)
        pass
    pass

In [ ]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000:
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

### optional (not verified): mask error

In [ ]:
#perepare error pack
error_pack = [[0]]
error_flag = 1
UnixInitials = np.array(time_category0[::capture_slice][inits], dtype=int)
for i in range(len(inits)):
    #print(f'{error_flag} {error_UnixTime[error_flag]} {UnixInitials[i]}')
    if (i < len(inits)-1) and (error_flag < len(error_UnixTime)-1):
        if (UnixInitials[i] <= error_UnixTime[error_flag]) and (error_UnixTime[error_flag] < UnixInitials[i+1]):
            error_pack[0].append(i)
            #print(f'{error_flag} {error_UnixTime[error_flag]}')
            error_flag += 1
            if (UnixInitials[i] <= error_UnixTime[error_flag]) and (error_UnixTime[error_flag] < UnixInitials[i+1]):
                error_flag += 1
                pass
            pass
        pass
    else:
        if (UnixInitials[i] <= error_UnixTime[error_flag]) and (error_UnixTime[error_flag] < UnixInitials[i]+min(np.diff(UnixInitials))):
            error_pack[0].append(i)
    pass

In [ ]:
error_sort = []
error_sort.append([error_pack[0][0]])
for i in sorted(error_pack[0][1:]):
    error_sort[0].append(i)
#error_sort

In [ ]:
#avoid error
error_position = []
error_flag0 = 0

error_flag2 = 0

for i in range(matrix_size):
    error_flag1 = 1
    error_position.append([])
    for j in range(len(inits)):
        if i == error_sort[error_flag0][0]:
            if error_flag1 <= len(error_sort[error_flag0])-1:
                if j == error_sort[error_flag0][error_flag1]:
                    #print(f'{error_sort[error_flag0][0]} {error_sort[error_flag0][error_flag1]}')
                    error_position[i].append([])
                    error_position[i][j] = False
                    error_flag1 += 1
                    error_flag2 = 1
                    pass
                else:
                    error_position[i].append([])
                    error_position[i][j] = True
                    pass
                pass
            else:
                error_position[i].append([])
                error_position[i][j] = True
                pass
            pass
        else:
            error_position[i].append([])
            error_position[i][j] = True
            pass
        pass
    if error_flag2 == 1:
        if error_flag0 != len(error_sort)-1:
            error_flag0 += 1
            error_flag2 = 0
            pass
        pass
    pass

In [ ]:
#cutting more
cp0 = reference > 60000
cp1 = time[:-1][~cp0[1:] & cp0[:-1]]-10
over_flag = 0

paku = []

for i in range(matrix_size):
    for j in range(len(inits)-1):
        if error_position[i][j] != False:
            cp2 = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]
            #print(f'{over_flag} {round(cp1[over_flag],2)} {round(cp2[0],2)} {round(cp2[-1],2)} {round(cp2[0],2) <=  round(cp1[over_flag],2)} {round(cp1[over_flag],2) < round(cp2[-1],2)}')
            if not (cp2[0] <=  cp1[over_flag]) and (cp1[over_flag] <= cp2[-1]):
                if over_flag < len(cp1)-1:
                    over_flag += 1
                    pass
            if (cp2[0] <=  cp1[over_flag]) and (cp1[over_flag] <= cp2[-1]):
                paku.append(j)
                if over_flag < len(cp1)-1:
                    over_flag += 1
                    pass
                pass
            pass
        pass
    pass

In [ ]:
for i in paku:
    error_pack[0].append(i)

In [ ]:
#avoid error
error_position = []
error_flag0 = 0

error_flag2 = 0

for i in range(matrix_size):
    error_flag1 = 1
    error_position.append([])
    for j in range(len(inits)):
        if i == error_sort[error_flag0][0]:
            if error_flag1 <= len(error_sort[error_flag0])-1:
                if j == error_sort[error_flag0][error_flag1]:
                    #print(f'{error_sort[error_flag0][0]} {error_sort[error_flag0][error_flag1]}')
                    error_position[i].append([])
                    error_position[i][j] = False
                    error_flag1 += 1
                    error_flag2 = 1
                    pass
                else:
                    error_position[i].append([])
                    error_position[i][j] = True
                    pass
                pass
            else:
                error_position[i].append([])
                error_position[i][j] = True
                pass
            pass
        else:
            error_position[i].append([])
            error_position[i][j] = True
            pass
        pass
    if error_flag2 == 1:
        if error_flag0 != len(error_sort)-1:
            error_flag0 += 1
            error_flag2 = 0
            pass
        pass
    pass

# Plots

In [ ]:
# so called speed
fig10 = plt.figure(figsize=(10,7))

cut_place = []

for i in range(matrix_size):
    cut_place.append([])
    for j in range(len(inits)-5):
        cut_place[i].append([])
        #if error_position[i][j] != False:
        cut_threshold = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]]
        cut_place[i][j] = min(np.where(cut_threshold >= 19)[0])
        plt.axvline(cut_threshold[cut_place[i][j]])
        plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
                    np.diff((reference_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])*Deg,label=f'num_{j}')
        #pass
        pass
    pass
plt.ylim([-1,4])

In [ ]:
moved_angle = []
end_position = []

fig21, ax = plt.subplots(figsize=(10,7))
plt.rcParams["font.size"] = 15
#ax = axarr.ravel()

for i in range(matrix_size):
    moved_angle.append([])
    end_position.append([])
    for j in range(iteration-1):
        #if error_position[i][j] != False:
        hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
        place = min(np.where(hoge >= 20)[0])
        moved_angle[i].append((lincount_block[i][block_initials[i][j] + place]-lincount_block[i][block_initials[i][j]])*Deg)
        end_position[i].append(lincount_block[i][block_initials[i][j] + place]%52000*Deg)
        #pass
        pass
    ax.scatter(end_position[i], moved_angle[i], color='b')
    pass

for i in range(len(np.arange(0,382.5,22.5))):
    ax.axvline(np.arange(0,382.5,22.5)[i], ls=':', color='r')
    pass

#plt.ylim([10,30])

ax.xaxis.set_major_locator(tik.MultipleLocator(45))

ax.set_title(f'Jan19_absolute2_feedback_8times')
ax.set_xlabel('position[deg]')
ax.set_ylabel('proceeded degree')

In [ ]:
x1 = np.arange(0,360,22.5)
x2 = np.roll(x1, 6)
x3 = np.tile(x2, 40)

In [ ]:
goal = end_position[0]

for i in np.where(np.array(end_position[0]) > 350)[0]:
    x3[i] = 360

In [ ]:
def linear_func(x,a,b):
    return a*x+b

In [ ]:
popt, pcov = curve_fit(linear_func, x3[:-1], goal)
popt

In [ ]:
fig30, axarr = plt.subplots(2,1,figsize=(9,12), gridspec_kw={'height_ratios':[3,1]})
plt.rcParams["font.size"] = 18
ax = axarr.ravel()
plt.subplots_adjust(hspace=0.)

piyo = np.linspace(0,360,len(x3[:-1]))

ax[0].scatter(x3[:-1], goal)
ax[0].plot(piyo, linear_func(piyo, popt[0], popt[1]),color='r', lw=1)

hoge = np.arange(0,382.5,22.5)
for i in range(len(hoge)):
    ax[0].axvline(hoge[i], ls=':', color='g')
    ax[0].axhline(hoge[i], ls=':', color='y')

#ax[0].set_xlabel('goal_position[deg]', fontsize=15)
ax[0].set_ylabel('stopped position[deg]')
ax[0].yaxis.set_major_locator(tik.MultipleLocator(45))
ax[0].legend(loc='upper left', title=f'gradient a={round(popt[0],5)}\nintercept b={round(popt[1],5)}')

ax[1].hist(goal, bins=360)
ax[1].set_xlabel('goal position[deg]')
ax[1].set_ylabel('number of events')
ax[1].xaxis.set_major_locator(tik.MultipleLocator(45))

In [ ]:
fig31, ax = plt.subplots(figsize=(10,7))
#ax = axarr.ravel()
plt.rcParams["font.size"] = 18

ax.xaxis.set_major_locator(tik.MultipleLocator(45))
                           
ax.scatter(x3[:-1], goal-x3[:-1], s=15)
ax.set_xlabel('goal position[deg]')
ax.set_ylabel('difference between goal and end position[deg]', fontsize=15)

In [ ]:
fig40 = plt.figure(figsize=(10,7))
plt.rcParams["font.size"] = 15
mu, std = norm.fit(moved_angle)

# Plot the histogram.
plt.hist(moved_angle, bins=40, density=True, alpha=0.3, color='g', label=f'feedback_8times')
plt.legend(fontsize=15,loc='upper right')

# Plot the PDF.
xmin, xmax = plt.xlim([10,35])
plt.ylim([0.,0.7])
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'g', linewidth=2)
plt.title(f'Fit Result: mu={round(mu,2)}, std={round(std,2)}')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('ratio')

## sub

In [ ]:
fig11, ax11 = plt.subplots(1,1,figsize=(300,7))

for i in range(matrix_size):
    for j in range(len(inits)-1):
        #if error_position[i][j] != False:
        ax11.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1],
                        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]),label=f'num_{j}')
            #pass
        pass
    pass

for i in range(len(error_Time)):
    ax11.axvline(error_Time[i], ls=':', lw=3)
    pass

fuga = reference > 60000
ax11.scatter((time[:-1])[~fuga[1:] & fuga[:-1]]-10, np.ones(len((time[:-1])[~fuga[1:] & fuga[:-1]]))*100)

ax11.set_ylim([-1,200])
#ax11.xlim([700,770])

ax11.xaxis.set_major_locator(tik.MultipleLocator(100))

ax11.set_title(f'Jan13_absolute_feedback_5times')
ax11.set_xlabel('time[sec]')
ax11.set_ylabel('difference of counts')

# Others